In [75]:
from oai_key import oai_key
from openai import OpenAI
from datetime import datetime
import pandas as pd


client = OpenAI(api_key=oai_key)

In [76]:
responses = []

for i in range(100):
    response = client.chat.completions.create(
      model="gpt-4o",
      messages=[
        {"role": "user", "content": """Alice has N brothers and she also has M sisters. How many sisters does Alice’s brother have?
         Write the final answer in the format Final Answer:<answer in terms of M and/or N>"""},
      ]
    )
    responses.append(response)
    if i % 10 == 0:
        print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} Completed {i+1} requests")

2024-06-20 23:26:49 Completed 1 requests
2024-06-20 23:27:12 Completed 11 requests
2024-06-20 23:27:32 Completed 21 requests
2024-06-20 23:27:52 Completed 31 requests
2024-06-20 23:28:15 Completed 41 requests
2024-06-20 23:28:41 Completed 51 requests
2024-06-20 23:29:01 Completed 61 requests
2024-06-20 23:29:25 Completed 71 requests
2024-06-20 23:29:49 Completed 81 requests
2024-06-20 23:30:12 Completed 91 requests


In [77]:
new_responses = []

for i in range(100):
    new_response = client.chat.completions.create(
      model="gpt-4o",
      messages=[
        {"role": "user", "content": """Here's a problem

Alice has N brothers and she also has M sisters. How many sisters does Alice’s brother have?

Can you expand the problem by adding missing details to this problem, e.g. make assumptions about M, and N, assign names, clearly state any implicit assumptions, etc.
Once you add details, answer the question 'How many sisters does Alice’s brother have?' and then generalize the answer in terms of M and N. Write the final answer in the format Final Answer:<answer in terms of M and/or N>"""},
      ]
    )
    new_responses.append(new_response)
    if i % 10 == 0:
        print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} Completed {i+1} requests")

2024-06-20 23:32:08 Completed 1 requests
2024-06-20 23:32:51 Completed 11 requests
2024-06-20 23:33:41 Completed 21 requests
2024-06-20 23:34:33 Completed 31 requests
2024-06-20 23:35:40 Completed 41 requests
2024-06-20 23:36:33 Completed 51 requests
2024-06-20 23:37:24 Completed 61 requests
2024-06-20 23:38:26 Completed 71 requests
2024-06-20 23:39:20 Completed 81 requests
2024-06-20 23:40:10 Completed 91 requests


In [79]:
# Create a DataFrame with the responses

responses_df = pd.DataFrame(
    {'prompt': ["""Alice has N brothers and she also has M sisters. How many sisters does Alice’s brother have?
         Write the final answer in the format Final Answer:<answer in terms of M and/or N>"""]* 100 + ["""Here's a problem

Alice has N brothers and she also has M sisters. How many sisters does Alice’s brother have?

Can you add details to this problem, e.g. make assumptions about M, and N, assign names, clearly state any implicit assumptions, etc.
Once you add details, answer the question 'How many sisters does Alice’s brother have?' and then generalize the answer in terms of M and N. Write the final answer in the format Final Answer:<answer>"""] * 100,
        'prompt_type': ['standard']*100 + ['expand_then_solve']*100,
        'response': [i.choices[0].message.content for i in responses] + [j.choices[0].message.content for j in new_responses]
    }
)

In [80]:
responses_df['final_answer'] = responses_df['response'].apply(lambda x: x.split('Final Answer:')[-1].strip() if len(x.split('Final Answer:')) > 1 else '')


In [81]:
responses_df['final_answer'].unique()

array(['M**', '} M\n\\]', '\\( M \\)', 'M', '\\( M + 1 \\)', '\\(M\\)',
       '} M \\]', '** \\( M \\)', '\\(M + 1\\)**', '\\(M + 1\\)',
       '\\(\\mathbf{M-1}\\)', '', '} M + 1\n\\]',
       '\\( M \\)\n\nHere, M represents the number of sisters Alice has. Therefore, each of Alice’s brothers also has M sisters.',
       '\\(M\\)**', '\\( M \\)**.',
       '\\(\\boxed{M}\\)\n\nThis means that Alice’s brother has \\(M\\) sisters.',
       '} M + 1 \\)',
       "** \\(M + 1\\)\n\nHere, we generalized the answer, showing that regardless of the specific values of \\( N \\) and \\( M \\), each of Alice's brothers will always have \\( M + 1 \\) sisters.",
       '\\( \\mathbf{M} \\)',
       'M\n\nThis means, irrespective of the number of brothers (N), each brother will have M sisters.',
       '} M + 1 \\]',
       '\\( M \\)\n\nThus, the generic form of the answer, regardless of specific values for M and N, remains the same.',
       '\\(M\\) (substituting the values for the appropriate

In [82]:
mapping_dict = {
    'M**': 'M',
    '} M\n\\]': 'M',
    '\\( M \\)': 'M',
    'M': 'M',
    '\\( M + 1 \\)': 'M+1',
    '\\(M\\)': 'M',
    '} M \\]': 'M',
    '** \\( M \\)': 'M',
    '\\(M + 1\\)**': 'M+1',
    '\\(M + 1\\)': 'M+1',
    '\\(\\mathbf{M-1}\\)': 'M-1',
    '': '',
    '} M + 1\n\\]': 'M+1',
    "\\( M \\)\n\nHere, M represents the number of sisters Alice has. Therefore, each of Alice's brothers also has M sisters.": 'M',
    '\\(M\\)**': 'M',
    '\\( M \\)**.': 'M',
    "\\(\\boxed{M}\\)\n\nThis means that Alice's brother has \\(M\\) sisters.": 'M',
    '} M + 1 \\)': 'M+1',
    "** \\(M + 1\\)\n\nHere, we generalized the answer, showing that regardless of the specific values of \\( N \\) and \\( M \\), each of Alice's brothers will always have \\( M + 1 \\) sisters.": 'M+1',
    '\\( \\mathbf{M} \\)': 'M',
    'M\n\nThis means, irrespective of the number of brothers (N), each brother will have M sisters.': 'M',
    '} M + 1 \\]': 'M+1',
    '\\( M \\)\n\nThus, the generic form of the answer, regardless of specific values for M and N, remains the same.': 'M',
    '\\(M\\) (substituting the values for the appropriate context).': 'M',
    '\\(M\\).': 'M',
    '** \\( M + 1 \\)': 'M+1',
    '**M**': 'M',
    '**M + 1**': 'M+1',
    '\\(M - 1\\)': 'M-1',
    '** M': 'M',
    "\\(M - 1\\)\n\nThe number of sisters that Alice's brother has is \\(M - 1\\).": 'M-1',
    '\\( \\text{M} \\)': 'M',
    '\\( M - 1 \\)': 'M-1',
    '\\( M \\).': 'M',
    'M + 1': 'M+1',
    '**\n\\[ \\boxed{M} \\]\n\nThis conclusion follows the understanding that Alice is included in the count of M sisters.': 'M',
    '\\( M + 1 \\)**.': 'M+1',
    "\\( M + 1 \\)\n\nTherefore, if Alice's brother has any number \\( N \\) of brothers and \\( M \\) sisters, he still counts Alice among his sisters, making the number of sisters \\( M + 1 \\).": 'M+1',
    "\\( M - 1 \\)\n\nIn this generalized form, irrespective of the specific names or number of brothers N, each of Alice's brothers will have \\( M - 1 \\) sisters, where M includes Alice herself.": 'M-1',
    '} M \\]\n\nThis means that each brother of Alice has \\( M \\) sisters.': 'M',
    "\\( M \\)\n\nIn terms of M, the number of sisters that Alice's brothers have is \\( M \\).": 'M',
    "\\( \\boxed{M} \\)\n\nSo, in general, Alice's brother has \\( M \\) sisters.": 'M',
    "** \\( M + 1 \\)\n\nSo, the number of sisters each of Alice's brothers has is \\( M + 1 \\).": 'M+1',
    "\\(M + 1\\)**\n\nSo in summary, each of Alice's brothers has \\(M + 1\\) sisters, where M is the number of sisters excluding Alice.": 'M+1',
    "Any one of Alice's brothers has M sisters.": 'M',
    "**\n\\[ \\text{Number of sisters Alice's brother has} = \\text{M} - 1 \\]\n\nSo, if Alice has N brothers and M sisters, each brother has \\(\\text{M} - 1\\) sisters.": 'M-1',
    '} M \\)': 'M',
    '\\( \\boxed{M} \\)': 'M',
    '** \\(M\\)': 'M',
    '\\( M + 1 \\) sisters.': 'M+1',
    "} M \\]\n\nSo, for any values of M and N:\n- An Alice's brother has M sisters.": 'M',
    "\\( \\text{M} + 1 \\)\n\nTherefore, if Alice has N brothers and M sisters, then each of Alice's brothers has \\( \\text{M} + 1 \\) sisters.": 'M+1',
    '**\n\\( M - 1 \\)': 'M-1',
    "} M + 1\n\\]\n\nThus, Alice's brother will have \\( M + 1 \\) sisters, accounting for all the female siblings in the family including Alice herself.": 'M+1',
    "\\( M \\)\n\n**Therefore, the number of sisters that Alice's brother has is \\( M \\).**": 'M'
}

In [83]:
responses_df['clean_answer'] = responses_df['final_answer'].map(mapping_dict)
responses_df['is_correct'] = responses_df['clean_answer'] == 'M+1'

In [84]:
responses_df.groupby('prompt_type')['is_correct'].sum()

prompt_type
expand_then_solve    28
standard             12
Name: is_correct, dtype: int64

In [85]:
responses_df.to_csv('data.csv', index=False)

To determine how many sisters Alice's brother has, we start by analyzing the given information:

- Alice has \( N \) brothers.
- Alice has \( M \) sisters.

For the problem, we need to identify how the number of sisters relates to Alice's brother. Here are the key points:

1. Alice herself is a sister.
2. Alice's sisters are the same as her brother's sisters (since they all share the same siblings in common).

Therefore, Alice's brother has the same sisters as Alice has.

So, the number of sisters that Alice's brother has is exactly equal to \( M \).

Final Answer: \( M \)
